We will scrap imdb movies and create the dataset

In [ ]:
# importing Libraries BeautifulSsoup,Scrapy,Selenium and requestsAX
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "https://www.imdb.com/chart/top/"
response = requests.get(url)
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
titles = soup.find_all('h3',class_='ipc-title__text')

In [ ]:
for i in titles:
  print(i.text.strip())

In [ ]:
years = soup.find_all('span',class_="sc-b4f120f6-7 hoOxkw cli-title-metadata-item")
for i in years:
  print(i.text.strip())

In [ ]:
Ratings = soup.find_all('span',class_="ipc-rating-star--rating")
for i in Ratings:
  print(i.text.strip())

In [ ]:
title = []
year = []
rating = []

# Fetch the page content using existing 'url' and 'headers' from kernel state
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# Find all individual movie entries using a more stable container class
movie_containers = soup.find_all('div', class_="sc-b4f120f6-0 bQhtuJ cli-children")

if not movie_containers:
    print(f"Warning: No movie containers found for URL: {url}. Please check the 'ipc-metadata-list-item' class or page structure.")
else:
    for movie_container in movie_containers:
        extracted_title = np.nan
        extracted_year = np.nan
        extracted_rating = np.nan

        # Extract title
        try:
            title_element = movie_container.find('h3', class_="ipc-title__text")
            if title_element:
                # Titles often have a number like "1. The Shawshank Redemption", strip that.
                extracted_title = title_element.text.strip().split('.', 1)[-1].strip()
        except Exception:
            pass

        # Extract year
        try:
            # The year is usually the first span with this metadata item class
            year_element = movie_container.find('span', class_="sc-b4f120f6-7 hoOxkw cli-title-metadata-item")
            if year_element:
                text_content = year_element.text.strip()
                # A simple check to ensure it looks like a year (4 digits)
                if text_content.isdigit() and len(text_content) == 4:
                    extracted_year = text_content
        except Exception:
            pass

        # Extract rating
        try:
            # The rating number is typically in a span with 'ipc-rating-star__rating' class
            rating_element = movie_container.find('span', class_="ipc-rating-star--rating")
            if rating_element:
                extracted_rating = rating_element.text.strip()
        except Exception:
            pass

        title.append(extracted_title)
        year.append(extracted_year)
        rating.append(extracted_rating)

In [ ]:
df = pd.DataFrame({'title':title,'year':year,'rating':rating})
df.to_csv('top_25_movies.csv',index=False)